# Sistema de asistencia basado en el reconocimiento facial

## Introducción

Este proyecto muestra un Sistema de asistencia basado en el reconocimiento facial desarrollado en Python. Los sistemas tradicionales de control de asistencia suelen ser manuales, propensos a errores y lentos. Aprovechando el reconocimiento facial, podemos automatizar el proceso, creando una solución más eficiente, precisa y rentable. Esto es especialmente beneficioso para instituciones como escuelas y empresas, donde el control de asistencia de un gran número de personas es una tarea diaria. La automatización elimina los errores humanos, evita las entradas fraudulentas (como la asistencia por poder) y libera un tiempo valioso para actividades más productivas.


## Antecedentes y motivación

Los sistemas manuales de control de asistencia adolecen de varios inconvenientes, como la posibilidad de cometer errores, el posible robo de tiempo o falsificación de registros asistenciales, y la carga administrativa que supone compilar manualmente los registros. Grandes empresas tecnológicas como Amazon, Microsoft y Face++ ya han demostrado el poder del reconocimiento facial para la seguridad, el control de acceso y la identificación de usuarios.

La motivación de este proyecto es aplicar conceptos fundamentales de la visión por ordenador y el aprendizaje automático para crear una aplicación práctica en el mundo real. El objetivo es crear un prototipo accesible y de bajo coste que pueda implementarse utilizando una cámara web estándar y bibliotecas Python de código abierto, convirtiéndolo en una opción viable para organizaciones con presupuestos limitados.


## Desarrollo del proyecto

### Herramientas utilizadas

* Lenguaje de programación:Python
* **Bibliotecas:**
    * `NumPy`: Para operaciones numéricas y manejo de datos de imagen como matrices.
    * `OpenCV`: Para procesamiento y manejo de imágenes.
    * `face_recognition`: Una librería sencilla y potente para detectar, reconocer y manipular caras.
    * `cmake`: Herramienta que configura y gestiona la compilación de bibliotecas como `dlib`.
    * `dlib`: Biblioteca que proporciona algoritmos para la deteción y codificación de rostros, fundamentales para el reconocimiento facial.
* **Hardware:** Ordenador portátil o de sobremesa con webcam (para captura de imágenes).
* **Entorno de desarrollo:** Cursor

### Paso 1: Instalar e importar las librerías necesarias

En primer lugar, tenemos que instalar las bibliotecas necesarias. `dlib` es un prerrequisito para `face_recognition`. También instalaremos `cmake` para asegurarnos de que `dlib` compila correctamente.

Adicionalmente, en caso de errores al tratar de instalar CMake, descargalo desde su sitio web oficial: https://cmake.org/download/. Abrí el instalador, marcá la opción "Add CMake to system PATH for all users", instalá, reiniciá la terminal y corré esta línea de código:


`cmake --version`

Deberías recibir un mensaje con la versión instalada de dicho sistema.

Por otro lado, en caso de tener errores en la instalación de dlib, descargá Visual Studio 2022, en su versión Community, desde https://visualstudio.microsoft.com/es/downloads/. Luego, en el instalador, seleccioná "Desarrollo para el escritorio con C++" y continuá con la instalación. Finalmente, reiniciá la terminal e intentá instalar las dependencias nuevamente.

In [1]:
!pip install requirements.txt

HINT: You are attempting to install a package literally named "requirements.txt" (which cannot exist). Consider using the '-r' flag to install the packages listed in requirements.txt


ERROR: Could not find a version that satisfies the requirement requirements.txt (from versions: none)
ERROR: No matching distribution found for requirements.txt


Ahora, vamos a importar las bibliotecas que vamos a utilizar en todo el proyecto.

In [2]:
import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime

### Paso 2: Cargar imágenes de personas conocidas

Necesitamos una base de datos de caras conocidas con las que comparar. Para este notebook, crearemos un directorio llamado `known_faces` y cargaremos imágenes en él. Cada archivo de imagen debe llevar el nombre de la persona que aparece en la foto (por ejemplo, `elon_musk.jpg`, `bill_gates.png`).

In [3]:
# --- PASO 2: VERIFICAR Y/O CREAR EL DIRECTORIO DE IMÁGENES ---

# Nombre del directorio para las imágenes de rostros conocidos.
# Lo definimos en una variable para usarlo fácilmente en todo el script.
image_directory = "IMG"

print("--- Verificando la configuración inicial ---")

# Comprobar si el directorio de imágenes no existe.
if not os.path.exists(image_directory):
    # Si no existe, notificar al usuario.
    print(f"El directorio '{image_directory}' no fue encontrado.")
    
    # Crear el directorio.
    os.makedirs(image_directory)
    print(f"Se ha creado el directorio '{image_directory}'.")
    
    # Dar la instrucción clave y detener el script para que el usuario pueda actuar.
    print("\n*** ACCIÓN REQUERIDA ***")
    print(f"Por favor, añade las imágenes de los rostros conocidos (.jpg, .png) en la carpeta '{image_directory}' y vuelve a ejecutar el script.")
    
    # Detenemos la ejecución. Sin imágenes, el resto del programa no puede funcionar.
    exit()
else:
    # Si el directorio ya existe, notificar que todo está en orden.
    print(f"Directorio '{image_directory}' encontrado. El programa continuará.")

--- Verificando la configuración inicial ---
Directorio 'IMG' encontrado. El programa continuará.


### Paso 3: Codificar 'Known Faces'

El núcleo del reconocimiento facial es convertir una cara en una representación matemática, llamada «codificación». Se trata de un vector de 128 números que es único para cada cara. Ahora procesaremos cada imagen cargada, detectaremos la cara y generaremos su codificación. Almacenaremos estas codificaciones junto con el nombre de la persona.

In [4]:
# --- PASO 3: CODIFICAR LOS ROSTROS CONOCIDOS ---
# 
# # Listas para almacenar las codificaciones de los rostros conocidos y sus nombres.
known_face_encodings = []
known_face_names = []

print("Procesando rostros conocidos...")

# Recorrer cada archivo en el directorio de rostros conocidos.
for filename in os.listdir(image_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        # Cargar el archivo de imagen.
        image_path = os.path.join(image_directory, filename)
        image = face_recognition.load_image_file(image_path)

        # Obtener la codificación del rostro (se asume una cara por imagen).
        face_encodings = face_recognition.face_encodings(image)

        if face_encodings:
            encoding = face_encodings[0]
            # Agregar la codificación y el nombre (sin la extensión del archivo) a las listas.
            known_face_encodings.append(encoding)
            known_face_names.append(os.path.splitext(filename)[0])

print(f"Se procesaron {len(known_face_names)} rostros conocidos.")
print("Nombres conocidos:", known_face_names)

Procesando rostros conocidos...
Se procesaron 6 rostros conocidos.
Nombres conocidos: ['Adrian_DIAZ', 'Ana_DEARMAS', 'Daiana_FRETE', 'Hanna_DIAZ', 'Michael_MANDO', 'Nataly_PEREZ']


### PASO 4: REGISTRAR LOS ROSTROS CONOCIDOS COMO ASISTENCIAS


Se registra la asistencia de una persona en el archivo `attendance.csv`, guardando su nombre y la hora exacta en que fue detectada. Antes de agregar una nueva entrada, verifica si esa persona ya fue registrada en la sesión actual, evitando duplicados. Usa el modo de archivo 'a+' para permitir lectura y escritura, y para crear el archivo si no existe.

In [5]:
# --- PASO 4: FUNCIÓN PARA MARCAR LA ASISTENCIA ---

def mark_attendance(name):
    """
    Registra el nombre y la hora en el archivo attendance.csv.
    Evita registrar a la misma persona más de una vez por sesión.
    """
    # Usar 'a+' para abrir el archivo en modo de anexo (lo crea si no existe).
    with open('attendance.csv', 'a+') as f:
        # Mover el cursor al inicio para leer las entradas existentes.
        f.seek(0)
        my_data_list = f.readlines()
        name_list = []
        for line in my_data_list:
            entry = line.split(',')
            name_list.append(entry[0])

        # Comprobar si el nombre no ha sido ya registrado.
        if name not in name_list:
            now = datetime.now()
            dt_string = now.strftime('%Y-%m-%d %H:%M:%S')
            f.writelines(f'\n{name},{dt_string}')
            print(f"Asistencia marcada para {name}")

### Paso 5: Capturar una Imagen para la Asistencia

Ahora es el momento de pasar lista. En una aplicación real, esto implicaría capturar un fotograma en directo de una cámara web.

In [6]:
# --- PASO 5: CAPTURAR VIDEO Y REALIZAR EL RECONOCIMIENTO FACIAL ---

# Iniciar la captura de video desde la cámara web (el índice 0 suele ser la cámara por defecto).
video_capture = cv2.VideoCapture(0)

print("\nIniciando cámara. Presiona 'q' para salir.")

while True:
    # Capturar un solo fotograma de video.
    ret, frame = video_capture.read()
    if not ret:
        print("Error: No se pudo capturar el fotograma.")
        break

    # Redimensionar el fotograma para un procesamiento más rápido (opcional).
    small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)

    # Convertir la imagen de BGR (OpenCV) a RGB (face_recognition).
    rgb_small_frame = cv2.cvtColor(small_frame, cv2.COLOR_BGR2RGB)

    # Encontrar todas las ubicaciones y codificaciones de rostros en el fotograma actual.
    face_locations = face_recognition.face_locations(rgb_small_frame)
    face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)

    # Recorrer cada rostro encontrado en el fotograma.
    for face_encoding, face_loc in zip(face_encodings, face_locations):
        # Comparar el rostro actual con la lista de rostros conocidos.
        matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
        name = "Desconocido"  # Nombre por defecto si no hay coincidencia.

        # Encontrar la mejor coincidencia usando la distancia euclidiana.
        face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
        best_match_index = np.argmin(face_distances)
        
        if matches[best_match_index]:
            name = known_face_names[best_match_index]
            # Llamar a la función para marcar la asistencia
            mark_attendance(name)

        # Dibujar un cuadro alrededor del rostro y mostrar el nombre.
        # Escalar las ubicaciones del rostro al tamaño original del fotograma.
        top, right, bottom, left = face_loc
        top *= 4
        right *= 4
        bottom *= 4
        left *= 4

        # Dibujar el rectángulo y la etiqueta con el nombre.
        cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
        cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
        font = cv2.FONT_HERSHEY_DUPLEX
        cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)

    # Mostrar la imagen resultante.
    cv2.imshow('Reconocimiento Facial - Asistencia (Presiona "q" para salir)', frame)

    # Salir del bucle si se presiona la tecla 'q'.
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Liberar recursos al finalizar
video_capture.release()
cv2.destroyAllWindows()


Iniciando cámara. Presiona 'q' para salir.



### Paso 6: Comprobar el registro de asistencia

Por último, vamos a ver el contenido de nuestro archivo `attendance.csv`. Este archivo registra el nombre de la persona reconocida y la fecha y hora en que se marcó su asistencia. Cada persona sólo se marca una vez por sesión.

In [8]:
# --- PASO 6: Comprobación de registros ---

print("\n--- Registro de Asistencia Final ---")
try:
    with open('attendance.csv', 'r') as f:
        print(f.read())
except FileNotFoundError:
    print("El archivo de asistencia no se creó. Nadie fue reconocido.")


--- Registro de Asistencia Final ---

Adrian_DIAZ,2025-06-22 19:21:22


### POST DATA --- Procesando un video

In [2]:
import face_recognition
import cv2
import numpy as np
import os
from datetime import datetime
from pathlib import Path

# Nombre del directorio para las imágenes de rostros conocidos.
image_directory_name = "IMGV"
image_path = Path(image_directory_name) # 1. Crear un objeto Path

print("--- Verificando la configuración inicial ---")

# 2. Usar los métodos del objeto Path. Es más legible.
if not image_path.is_dir(): 
    print(f"El directorio '{image_path}' no fue encontrado.")
    
    # 3. Crear el directorio. parents=True es para crear carpetas anidadas si fuera necesario.
    image_path.mkdir(parents=True, exist_ok=True) 
    
    # 4. Mensaje consolidado para el usuario.
    print(f"Se ha creado el directorio '{image_path}'.\n")
    print("*" * 25)
    print("ACCIÓN REQUERIDA:")
    print(f"Por favor, añade las imágenes de los rostros (.jpg, .png) en esta carpeta y vuelve a ejecutar el script.")
    print("*" * 25)
    
    exit() # Se mantiene la lógica de salida, que es fundamental.
else:
    print(f"Directorio '{image_path}' encontrado. El programa continuará.")


known_face_encodings = []
known_face_names = []

print("Procesando rostros conocidos...")
for filename in os.listdir(image_directory):
    if filename.endswith(('.jpg', '.jpeg', '.png')):
        image_path = os.path.join(image_directory, filename)
        image = face_recognition.load_image_file(image_path)
        face_encodings_list = face_recognition.face_encodings(image)
        if face_encodings_list:
            encoding = face_encodings_list[0]
            known_face_encodings.append(encoding)
            known_face_names.append(os.path.splitext(filename)[0])
print(f"Se procesaron {len(known_face_names)} rostros.")

# --- FUNCIÓN PARA MARCAR LA ASISTENCIA (Sin cambios) ---

def mark_attendance(name):
    with open('attendance_video.csv', 'a+') as f:
        f.seek(0)
        my_data_list = f.readlines()
        name_list = [line.split(',')[0] for line in my_data_list]
        if name not in name_list:
            now = datetime.now()
            dt_string = now.strftime('%Y-%m-%d %H:%M:%S')
            f.writelines(f'\n{name},{dt_string}')
            print(f"Rostro de {name} identificado y registrado.")

# --- PASO 2: PROCESAR EL VIDEO ---

# --- PASO 3: PROCESAR TODOS LOS VIDEOS DE LA CARPETA 'video'
video_folder = 'VideoIn'
output_folder = 'VideoOut'
os.makedirs(output_folder, exist_ok=True)

video_files = [f for f in os.listdir(video_folder) if f.lower().endswith(('.mp4', '.avi', '.mov'))]

for video_file in video_files:
    input_video_path = os.path.join(video_folder, video_file)
    output_video_path = os.path.join(output_folder, f'procesado_{video_file}')

    video_capture = cv2.VideoCapture(input_video_path)
    if not video_capture.isOpened():
        print(f"Error: No se pudo abrir el video {video_file}")
        continue

    frame_width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = int(video_capture.get(cv2.CAP_PROP_FPS))
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    video_writer = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    print(f"\nProcesando video: {video_file}")

    while video_capture.isOpened():
        ret, frame = video_capture.read()
        if not ret:
            break

        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_encoding, face_loc in zip(face_encodings, face_locations):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            name = "Desconocido"
            face_distances = face_recognition.face_distance(known_face_encodings, face_encoding)
            best_match_index = np.argmin(face_distances)
            if matches[best_match_index]:
                name = known_face_names[best_match_index]
                mark_attendance(name)

            top, right, bottom, left = face_loc
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6),
                        cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)

        video_writer.write(frame)

    video_capture.release()
    video_writer.release()
    print(f"✔ Video guardado: {output_video_path}")

cv2.destroyAllWindows()
print("\nTodos los videos fueron procesados correctamente.")

--- Verificando la configuración inicial ---
Directorio 'IMGV' encontrado. El programa continuará.
Procesando rostros conocidos...
Se procesaron 6 rostros.

Procesando video: Best_of_Glambot_2025.mp4


KeyboardInterrupt: 

In [5]:
import face_recognition
import cv2
import numpy as np
from pathlib import Path
from datetime import datetime
import os
import csv

# --- CONFIGURACIÓN DE RUTAS ---
image_path = Path("IMGV")
video_folder = Path("VideoIn")
output_folder = Path("VideoOut")
output_folder.mkdir(exist_ok=True)

print("--- Verificando la configuración inicial ---")

# --- CREAR DIRECTORIO PARA IMÁGENES SI NO EXISTE ---
if not image_path.is_dir():
    image_path.mkdir(parents=True, exist_ok=True)
    print(f"\nSe ha creado la carpeta '{image_path}'.")
    print("*" * 25)
    print("ACCIÓN REQUERIDA:")
    print("Agrega imágenes (.jpg/.png) de rostros conocidos en esa carpeta.")
    print("*" * 25)
    exit()

print(f"Directorio '{image_path}' encontrado. Continuando...\n")

# --- CARGAR ROSTROS CONOCIDOS ---
known_face_encodings = []
known_face_names = []

print("Procesando rostros conocidos...")

for img_file in image_path.glob("*.[jp][pn]g"):  # .jpg, .jpeg, .png
    image = face_recognition.load_image_file(img_file)
    encodings = face_recognition.face_encodings(image)
    if encodings:
        known_face_encodings.append(encodings[0])
        known_face_names.append(img_file.stem)

print(f"✔ Se procesaron {len(known_face_names)} rostros conocidos.")

# --- FUNCION PARA REGISTRAR ASISTENCIA ---
attendance_file = Path("attendance_video.csv")
attendance_set = set()

if attendance_file.exists():
    with attendance_file.open() as f:
        reader = csv.reader(f)
        attendance_set = {row[0] for row in reader if row}

def mark_attendance(name):
    if name not in attendance_set:
        now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        with attendance_file.open("a", newline="") as f:
            writer = csv.writer(f)
            writer.writerow([name, now])
        attendance_set.add(name)
        print(f"✔ Rostro identificado: {name} (asistencia registrada)")

# --- PROCESAR VIDEOS ---
video_files = [f for f in video_folder.glob("*") if f.suffix.lower() in ['.mp4', '.avi', '.mov']]

if not video_files:
    print("⚠ No se encontraron videos en la carpeta VideoIn.")
    exit()

for video_file in video_files:
    print(f"\nProcesando video: {video_file.name}")
    cap = cv2.VideoCapture(str(video_file))

    if not cap.isOpened():
        print(f"❌ No se pudo abrir: {video_file.name}")
        continue

    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS) or 24  # fallback a 24 fps
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')

    output_path = output_folder / f"procesado_{video_file.name}"
    out = cv2.VideoWriter(str(output_path), fourcc, fps, (frame_width, frame_height))

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        face_locations = face_recognition.face_locations(rgb)
        encodings = face_recognition.face_encodings(rgb, face_locations)

        for encoding, loc in zip(encodings, face_locations):
            matches = face_recognition.compare_faces(known_face_encodings, encoding)
            name = "Desconocido"

            if matches:
                distances = face_recognition.face_distance(known_face_encodings, encoding)
                best_match = np.argmin(distances)
                if matches[best_match]:
                    name = known_face_names[best_match]
                    mark_attendance(name)

            top, right, bottom, left = loc
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 255, 0), cv2.FILLED)
            cv2.putText(frame, name, (left + 6, bottom - 6),
                        cv2.FONT_HERSHEY_DUPLEX, 1.0, (255, 255, 255), 1)

        out.write(frame)

    cap.release()
    out.release()
    print(f"✔ Video guardado como: {output_path.name}")

cv2.destroyAllWindows()
print("\n✅ Todos los videos fueron procesados exitosamente.")


--- Verificando la configuración inicial ---
Directorio 'IMGV' encontrado. Continuando...

Procesando rostros conocidos...
✔ Se procesaron 6 rostros conocidos.

Procesando video: Best_of_Glambot_2025.mp4


KeyboardInterrupt: 